## 1. Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime

## 2. Loading Dataset

In [3]:
# Loading Dataset train and Test
df_train=pd.read_csv("train_users.csv")
df_test=pd.read_csv("test_users.csv")

In [5]:
df_test['country_destination']='TEST'

In [6]:
df_test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,TEST
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,TEST
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,TEST
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,TEST
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,TEST


In [7]:
# Loading Dataset of user session
df_session=pd.read_csv("sessions.csv")

In [8]:
df = pd.concat((df_train, df_test),axis=0, sort=False)

In [9]:
df_train.shape

(213451, 16)

In [10]:
df_test.shape

(62096, 16)

In [13]:
df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


## 3.  Preprocessing

### 3.1 Preprocessing of Session Dataset

In [14]:
df_session.shape
print("the session dataset is made of 10567737 rows and 6 columns")

the session dataset is made of 10567737 rows and 6 columns


In [15]:
# Count the null variables of the session dataset
display(df_session.isnull().sum())

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [16]:
df_session.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [17]:
#replace the unknow columns with null values
df_session['action_detail'].replace('-unknown-', np.nan, inplace=True)
df_session['action'].replace('-unknown-', np.nan, inplace=True)
df_session['device_type'].replace('-unknown-', np.nan, inplace=True)
df_session['secs_elapsed'].replace('-unknown-', np.nan, inplace=True)
df_session['action_type'].replace('-unknown-', np.nan, inplace=True)

In [18]:
#drops the null values
df_session.dropna(subset=['action_detail'], inplace=True)
df_session.dropna(subset=['action'], inplace=True)
df_session.dropna(subset=['device_type'], inplace=True)
df_session.dropna(subset=['secs_elapsed'], inplace=True)
df_session.dropna(subset=['action_type'], inplace=True)

In [19]:
#rename the culumns "user_id" with "id"
df_session.rename(columns={'user_id': 'id'}, inplace=True)

In [20]:
# I sum the seconds that each user spends on a singol action, the action_type, the action_detail, and the device_type.
# Then I transforms the values of this features in columns by unstack
df_action= df_session.groupby(['id', 'action'])['secs_elapsed'].agg(len).unstack()
df_action_type= df_session.groupby(['id', 'action_type'])['secs_elapsed'].agg(len).unstack()
df_action_detail= df_session.groupby(['id', 'action_detail'])['secs_elapsed'].agg(len).unstack()
df_device_type= df_session.groupby(['id', 'device_type'])['secs_elapsed'].agg(sum).unstack()

session_pivot=pd.concat([df_action,df_action_type,df_action_detail,df_device_type], axis=1)

#I add the string "count" for each name of the columns
session_pivot.columns = session_pivot.columns.map(lambda x: str(x) + '_count')

session_pivot.index.names = ['id']
session_pivot.reset_index(inplace=True)

In [21]:
# this is the shape of the table session_pivot
session_pivot.shape

(130658, 297)

In [22]:
#there are not duplicates in the id user
session_pivot['id'].nunique()

130658

### 3.2 Preprocessing of total Dataset

In [23]:
df.shape
print("the total dataset (train+test) is made of 275547 rows and 16 columns")

the total dataset (train+test) is made of 275547 rows and 16 columns


In [24]:
# null values
display(df.isnull().sum())
print("Variables that contains null are: date_first_booking, age, first_affiliate_tracked")

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         186639
gender                          0
age                        116866
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

Variables that contains null are: date_first_booking, age, first_affiliate_tracked


In [25]:
# I drop all the country_destination that are NDF (not classified)
df_no_NDF=df[df['country_destination']!='NDF']

In [26]:
#I drop the columns "date_first_booking"
df_no_NDF.drop('date_first_booking', axis=1, inplace=True)

C:\Users\Laura.Taddei\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
df_no_NDF.reset_index(drop=True)

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
1,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
2,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
3,osr2jwljor,2010-01-01,20100101215619,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
4,lsw9q7uk0j,2010-01-02,20100102012558,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150999,cv0na2lf5a,2014-09-30,20140930235232,-unknown-,31.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE,TEST
151000,zp8xfonng8,2014-09-30,20140930235306,-unknown-,NaN,basic,23,ko,direct,direct,untracked,Android,Android Phone,-unknown-,TEST
151001,fa6260ziny,2014-09-30,20140930235408,-unknown-,NaN,basic,0,de,direct,direct,linked,Web,Windows Desktop,Firefox,TEST
151002,87k0fy4ugm,2014-09-30,20140930235430,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,TEST


In [28]:
df_no_NDF.isnull().sum()

id                             0
date_account_created           0
timestamp_first_active         0
gender                         0
age                        49252
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel              0
affiliate_provider             0
first_affiliate_tracked     1758
signup_app                     0
first_device_type              0
first_browser                  0
country_destination            0
dtype: int64

In [29]:
#replace the unknow columns with null values
df_no_NDF['gender'].replace("-unknown-", np.nan, inplace=True)
df_no_NDF['first_browser'].replace("-unknown-", np.nan, inplace=True)

C:\Users\Laura.Taddei\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [30]:
#set to null all the values of the age >95 and < 18
df_no_NDF.loc[df_no_NDF['age']>95]=np.nan
df_no_NDF.loc[df_no_NDF['age']<18]=np.nan

C:\Users\Laura.Taddei\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [31]:
#drop all the null values
df_no_NDF.dropna(subset=['gender'], inplace=True)
df_no_NDF.dropna(subset=['age'], inplace=True)
df_no_NDF.dropna(subset=['first_affiliate_tracked'], inplace=True)

C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pan

In [33]:
#Dates

#1. transforms the "date_account_created", "date_first_booking", "timestamp_first_active" in dates
df_no_NDF['date_account_created'] = pd.to_datetime(df_no_NDF['date_account_created'])
df_no_NDF['timestamp_first_active'] = pd.to_datetime((df_no_NDF.timestamp_first_active // 1000000), format='%Y%m%d')

C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [34]:
#2. Define a new variable "time_interval" which is the difference between the two dates in days
df_no_NDF['time_interval'] = ((df_no_NDF['date_account_created'] - df_no_NDF['timestamp_first_active'])).dt.days

C:\Users\Laura.Taddei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
df_no_NDF=df_no_NDF.reset_index(drop=True)

In [36]:
df_no_NDF.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,time_interval
0,4ft3gnwmtx,2010-09-28,2009-06-09,FEMALE,56.0,basic,3.0,en,direct,direct,untracked,Web,Windows Desktop,IE,US,476
1,bjjt8pjhuk,2011-12-05,2009-10-31,FEMALE,42.0,facebook,0.0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,765
2,lsw9q7uk0j,2010-01-02,2010-01-02,FEMALE,46.0,basic,0.0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,0
3,0d01nltbrs,2010-01-03,2010-01-03,FEMALE,47.0,basic,0.0,en,direct,direct,omg,Web,Mac Desktop,Safari,US,0
4,a1vcnhxeij,2010-01-04,2010-01-04,FEMALE,50.0,basic,0.0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,0


### 3.3 Merge the two Datasets

In [37]:
#I do a left join from the train Dataset with the session Dataset
df_merge_session=pd.merge(df_no_NDF,session_pivot,on='id',how='left')

In [38]:
df_merge_session.shape

(82897, 312)

In [39]:
# id columns has no duplicates
df_merge_session['id'].nunique()

82897

In [40]:
#List of columns
df_merge_session.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'gender', 'age',
       'signup_method', 'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider',
       ...
       'Chromebook_count', 'Linux Desktop_count', 'Mac Desktop_count',
       'Opera Phone_count', 'Tablet_count', 'Windows Desktop_count',
       'Windows Phone_count', 'iPad Tablet_count', 'iPhone_count',
       'iPodtouch_count'],
      dtype='object', length=312)

In [41]:
df_merge_session.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,Chromebook_count,Linux Desktop_count,Mac Desktop_count,Opera Phone_count,Tablet_count,Windows Desktop_count,Windows Phone_count,iPad Tablet_count,iPhone_count,iPodtouch_count
0,4ft3gnwmtx,2010-09-28,2009-06-09,FEMALE,56.0,basic,3.0,en,direct,direct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bjjt8pjhuk,2011-12-05,2009-10-31,FEMALE,42.0,facebook,0.0,en,direct,direct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,lsw9q7uk0j,2010-01-02,2010-01-02,FEMALE,46.0,basic,0.0,en,other,craigslist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0d01nltbrs,2010-01-03,2010-01-03,FEMALE,47.0,basic,0.0,en,direct,direct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a1vcnhxeij,2010-01-04,2010-01-04,FEMALE,50.0,basic,0.0,en,other,craigslist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# List of categorical Features
cat_features = ['gender', 'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

# transforms all the categorical features in dummy variables
df_model = pd.get_dummies(df_merge_session, columns=cat_features)

In [43]:
df_model.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'age',
       'signup_flow', 'country_destination', 'time_interval', '10_count',
       '11_count', '12_count',
       ...
       'first_browser_Safari', 'first_browser_SeaMonkey', 'first_browser_Silk',
       'first_browser_SiteKiosk', 'first_browser_Sogou Explorer',
       'first_browser_Stainless', 'first_browser_TenFourFox',
       'first_browser_TheWorld Browser', 'first_browser_UC Browser',
       'first_browser_Yandex.Browser'],
      dtype='object', length=415)

In [44]:
df_model.head()

,id,date_account_created,timestamp_first_active,age,signup_flow,country_destination,time_interval,10_count,11_count,12_count,...,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser
0,4ft3gnwmtx,2010-09-28,2009-06-09,56.0,3.0,US,476,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,bjjt8pjhuk,2011-12-05,2009-10-31,42.0,0.0,other,765,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,lsw9q7uk0j,2010-01-02,2010-01-02,46.0,0.0,US,0,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
3,0d01nltbrs,2010-01-03,2010-01-03,47.0,0.0,US,0,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
4,a1vcnhxeij,2010-01-04,2010-01-04,50.0,0.0,US,0,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0


## 4. The ML model

In [46]:
# Total dataset
df_model.shape

(82897, 415)

### 4.1 Define Train and Test Datasets

In [47]:
df_train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [48]:
df_model.shape

(82897, 415)

In [50]:
#test dataset
test=df_model[df_model['country_destination']=='TEST']
test.drop('country_destination', axis=1, inplace=True)
test.reset_index(drop=True)

In [56]:
train=df_model[df_model['country_destination']!='TEST']
train.reset_index(drop=True)

In [77]:
#y_train=output values
y_train=train['country_destination']

#drop the columns with date, id, target
train.drop(['country_destination','id','date_account_created','timestamp_first_active'], axis=1, inplace=True)

#X_train= input values for the model
X_train=train.values

In [79]:
#drop the columns with date, id, target
test.drop(['id','date_account_created','timestamp_first_active'], axis=1, inplace=True)

#X_test= input values for the model
X_test=test.values

In [80]:
test.shape

(27548, 411)

### XGBoost model

In [81]:
# I use label encoder to convert the string values of y_train to numerical values
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [82]:
y_train_encoded

array([ 9, 10,  9, ...,  3,  9,  9])

In [83]:
#Dmatrix
import xgboost as xgb

xgtrain = xgb.DMatrix(X_train, label=y_train_encoded)

In [113]:
xg_clas = xgb.XGBClassifier(objective ='multi:softprob', colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 5, n_estimators = 10)

xg_clas.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [114]:
pred = xg_clas.predict(X_test)

In [115]:
pd.Series(pred).value_counts()

US       27509
FR          25
NL          11
ES           1
other        1
IT           1
dtype: int64

## 5. Definition of the score

In [107]:
def ndcg_score(preds, dtrain):
    labels = dtrain.get_label()
    top = []

    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])

    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg', score

## 6. Cross Validation

In [116]:
param = {
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 5,
    'objective': 'multi:softprob',
    'num_class': 12,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'nthread': 4,
    'seed': 42
}
# Do cross validation
num_round = 5
xgb.cv(param, xgtrain, num_boost_round=num_round, metrics=['mlogloss'], feval=ndcg_score)

,train-mlogloss-mean,train-mlogloss-std,train-ndcg-mean,train-ndcg-std,test-mlogloss-mean,test-mlogloss-std,test-ndcg-mean,test-ndcg-std
0,2.206301,0.000995,0.830001,0.000802,2.207704,0.000887,0.826822,0.001623
1,2.026138,0.001511,0.830089,0.000887,2.028864,0.001502,0.827077,0.001737
2,1.892733,0.001830,0.830498,0.000645,1.896700,0.001943,0.827411,0.001581
3,1.787790,0.002091,0.830798,0.000651,1.792870,0.002386,0.827502,0.001687
4,1.702207,0.002304,0.831046,0.000655,1.708441,0.002757,0.827443,0.001658


The best value of the test-score that I obtained is 0.827443